In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [2]:
def get_soup(url):
    # Make a GET request to the website
    response = requests.get(
        url,
        headers={'User-Agent': 'Mozilla/5.0'}
    )

    return BeautifulSoup(response.content, "lxml")

In [3]:
season_pages = {
    2020:'https://2kdb.net/collections/20',
    2021:'https://2kdb.net/collections/21',
    2022:'https://2kdb.net/collections/22',
    2023:'https://2kdb.net/collections/23',
}

In [6]:
verbose = True

rating_all_players = []
for season_year, url in season_pages.items():

    if verbose: print(season_year)


    soup = get_soup(url)

    team_links = soup.find_all('a', {"class": 'hover:bg-navbar rounded-xl p-4'})

    for link in team_links:
        url_team = 'https://2kdb.net'+link['href']
        team_name = link['title']
        
        if verbose: print(team_name)

        # Make a GET request to the website
        soup_team = get_soup(url_team)
        players_team = soup_team.find_all('a')
        for player in players_team:
            if (re.search('player',player['href']) is not None and player.text==''):
                time.sleep(1)
                url_player = 'https://2kdb.net'+player['href']

                soup_player = get_soup(url_player)
                # print(soup_player)
                name = soup_player.find('span',{'class':'flex items-center gap-2 font-bold text-3xl'}).text
                player_name = re.split('\d',name)[0]
                
                if verbose: print(player_name)

                stats = soup_player.find_all('div',{'class':'stats'})

                rating_dict = {}
                rating_dict['Season Year'] = season_year
                rating_dict['Team'] = team_name
                rating_dict['Player'] = player_name
                for rating_whole in stats:
                    rating_list = rating_whole.find_all('span')
                    # print(rating_list[0].text,rating_list[1].text)
                    rating_dict[rating_list[1].text] = rating_list[0].text
                rating_all_players.append(rating_dict)


2020
Atlanta Hawks
Trae Young
John Collins
Jabari Parker
Kevin Huerter
De'Andre Hunter
Alex Len
Cameron Reddish
Damian Jones
Evan Turner
Chandler Parsons
DeAndre Bembry
Allen Crabbe
Bruno Fernando
Brandon Goodwin
Boston Celtics
Kemba Walker
Jayson Tatum
Marcus Smart
Gordon Hayward
Enes Kanter
Jaylen Brown
Daniel Theis
Robert Williams
Semi Ojeleye
Vincent Poirier
Brad Wanamaker
Romeo Langford
Grant Williams
Carsen Edwards
Brooklyn Nets
Kevin Durant
Kyrie Irving
DeAndre Jordan
Spencer Dinwiddie
Jarrett Allen
Joe Harris
Caris LeVert
Taurean Prince
Wilson Chandler
Rodions Kurucs
Garrett Temple
David Nwaba
Nicolas Claxton
Dzanan Musa
Theo Pinson
Charlotte Hornets
Terry Rozier III
Nicolas Batum
Miles Bridges
Marvin Williams
Cody Zeller
Michael Kidd-Gilchrist
Willy Hernangomez
Bismack Biyombo
Malik Monk
Dwayne Bacon
P.J. Washington
Devonte Graham
Chicago Bulls
Zach LaVine
Lauri Markkanen
Otto Porter
Thaddeus Young
Wendell Carter Jr.
Coby White
Kris Dunn
Tomas Satoransky
Denzel Valentine
Chand

In [7]:
pd.DataFrame(rating_all_players).to_csv('ratings_NBA2k_2020_2023.csv')

pd.DataFrame(rating_all_players).head()

,Season Year,Team,Player,Shot Close,Shot Mid,Shot 3pt,Shot IQ,Free Throw,Offensive Consistency,Driving Layup,...,Help Defense IQ,Lateral Quickness,Pass Perception,Steal,Block,Defensive Consistency,Offensive Rebound,Defensive Rebound,Intangibles,Potential
0,2020,Atlanta Hawks,Trae Young,83,77,80,90,81,96,85,...,56,68,68,39,35,28,36,48,98,89
1,2020,Atlanta Hawks,John Collins,69,67,72,90,70,89,76,...,68,51,32,26,51,54,80,75,95,86
2,2020,Atlanta Hawks,Jabari Parker,81,70,69,90,66,75,82,...,64,62,63,36,50,50,44,75,95,78
3,2020,Atlanta Hawks,Kevin Huerter,72,70,80,90,71,63,75,...,61,70,62,41,49,33,37,47,95,82
4,2020,Atlanta Hawks,De'Andre Hunter,62,71,74,90,70,42,66,...,72,80,52,32,46,72,47,51,95,88
